In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import os

In [2]:
max_dataset_size = 3000000
waveform_length = 72;
nb_of_elements = 30000
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
root_folder = os.path.dirname(os.getcwd())
path_to_recording= path.join(root_folder,'data/recording_datasets/datasets_single9.npy');
path_to_ground_truth_data = path.join(root_folder,'data/recording_datasets/single_recording_gd9.npy');
path_to_train_data = path.join(root_folder,'data/train_data.npy');
path_to_train_labels = path.join(root_folder,'data/train_labels.npy');
path_to_noise_data = path.join(root_folder,'data/noise_data.npy');
path_to_mean_std = path.join(root_folder,'data/mean_std.npy');
sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [4]:
noise_indices = cnn.GetNoiseIndices(path_to_recording, path_to_ground_truth_data, waveform_length, nb_of_elements);



In [ ]:
noise_class = torch.zeros(1, noise_indices.nelement(), dtype= torch.int);
noise_data = torch.cat((noise_indices, noise_class), 0);


In [ ]:
np.save(path_to_noise_data, noise_data.numpy());

In [ ]:
%pdb on
dataset_spikes = cnn.GenerateDataset(path_to_recording, path_to_ground_truth_data, waveform_length, max_dataset_size);


Automatic pdb calling has been turned ON
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  2
shift_indexes:  tensor([-18, -16, -14, -12, -10,  -8,  -6,  -4,  -2,   0,   2,   4,   6,   8,
         10,  12,  14,  16,  18], dtype=torch.int32)
snr_ratio:  None
flip_data_horz:  1
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
started movingmeanandstd


In [ ]:
dataset_noise = cnn.GenerateDataset(path_to_recording, path_to_noise_data, waveform_length, max_dataset_size);


In [ ]:
dataset_size =  dataset_spikes.__len__() if dataset_spikes.__len__() < dataset_noise.__len__() else  dataset_noise.__len__()

In [ ]:
# convers to np array
np_data_spikes = np.zeros((dataset_size, 1, waveform_length));
for i, (data, target) in enumerate(dataset_spikes):
  np_data_spikes[i, :] = data.numpy();
  if(i + 1 == dataset_size):
    break;
    
del dataset_spikes

# removes spikes that have mutiple spikes in waveform
spike_argmax = np.argmax(np_data_spikes, 2)
over_treshold = waveform_length // 2 + waveform_length // 4
under_treshold = waveform_length // 2 - waveform_length // 4
is_valid_train_data = ((spike_argmax >= under_treshold) & (spike_argmax <= over_treshold)).ravel()
np_data_spikes = np_data_spikes[is_valid_train_data, :, :]
dataset_size = np_data_spikes.shape[0]
np_classes_spikes = np.ones(dataset_size)


np_data_noise = np.zeros((dataset_size, 1, waveform_length));
np_classes_noise = np.zeros(dataset_size)
for i, (data, target) in enumerate(dataset_noise):
  np_data_noise[i, :] = data.numpy();
  if(i + 1 == dataset_size):
    break;
    
del dataset_noise

    


In [ ]:
np_data_noise.shape

In [ ]:
dataset = np.concatenate((np_data_spikes, np_data_noise), 0);

In [ ]:
labels = np.concatenate((np_classes_spikes, np_classes_noise), 0); 

In [ ]:
np.save(path_to_train_data, dataset)
np.save(path_to_train_labels, labels)


In [ ]:
mean = (np.mean(np_data_spikes))
std = (np.std(np_data_spikes))
np.save(path_to_mean_std, [mean, std])

print(mean)
print(std)

In [ ]:
import matplotlib.pyplot as plt
rnd = np.random.randint(0, np_data_spikes.shape[0]);
plt.plot(np_data_spikes[rnd, 0, :])
np.argmax(np_data_spikes[rnd, 0, :])